In [4]:
#pip install langchain

In [6]:
#pip install openai

In [8]:
#pip install langchainhub

In [11]:
#pip install google-search-results

In [12]:
#!pip install duckduckgo-search

In [13]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain import hub
from langchain.agents import load_tools,Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chains import LLMMathChain, LLMChain

In [45]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GOOGLE_CSE_ID")
secret_value_1 = user_secrets.get_secret("GOOGLE_SEARCH_API_KEY")
secret_value_2 = user_secrets.get_secret("OPEN_API_KEY")
secret_value_3 = user_secrets.get_secret("SerpAPI")

In [15]:
llm=ChatOpenAI(openai_api_key=secret_value_0,model_name="gpt-3.5-turbo-0125",temperature=0)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
os.environ["SERPER_API_KEY"]=secret_value_1

In [46]:

os.environ["GOOGLE_CSE_ID"] = secret_value_0
os.environ["GOOGLE_API_KEY"] = secret_value_1

In [34]:
google_search = GoogleSearchAPIWrapper()

In [60]:
tools=[
   Tool(
       name="Search",
       func=google_search.run,
       description="useful for when you need to ask with search",
   )
]


In [61]:
problem_chain = LLMMathChain.from_llm(llm=llm)
tools.append(Tool(name="Calculator",
                func=problem_chain.run,
                 description="Useful for when you need to answer questions about math. This tool is used for calculation perpose. Only input math expressions."))

In [62]:
prompt = hub.pull("hwchase17/react")

In [63]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [64]:
#agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


In [65]:
agent = create_react_agent(llm, tools, prompt)

In [66]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [67]:
agent_executor.invoke({"input":"Which hotel is closest to the most popular beach in Goa, and How much it cost per night if it has 50% discount?"})



> Entering new AgentExecutor chain...
I need to search for the most popular beach in Goa and then find the closest hotel to that beach.
Action: Search
Action Input: Most popular beach in GoaResults 1 - 30 of 84 ... It is situated south of the busy Colva beach. Benaulim beach has quite a good number of shacks. Good fresh food is offered daily. It is a ... Results 1 - 30 of 84 ... Benaulim beach has quite a good number of shacks. Good fresh food is offered daily. It is a clean beach & has water sports aswell. Most ... Baga beach takes the crown of the most popular beach in Goa. It is alive throughout the day and well into the night. Lined with restaurants, shacks, clubs and ... Jun 5, 2020 ... A mile-long spotless crescent-moon beach. Lazy days under the sun with cheap cocktails and a good book. Great curries for lunch and dinner. Oct 8, 2014 ... 1. Palolem Beach:- Visited mainly by the fisherman and foreign tourists, Palolem (In spite of being among the most famous beaches in Goa) get

{'input': 'Which hotel is closest to the most popular beach in Goa, and How much it cost per night if it has 50% discount?',
 'output': 'Rs. 2457.50 per night'}